In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_fscore_support
from sklearn.preprocessing import label_binarize
from sklearn.feature_selection import SelectKBest, f_classif  # f_classif 사용
from scipy.stats import randint as sp_randint

In [36]:
# 1. 데이터 불러오기
data = pd.read_csv('azdigar nav sartirovka+0 delete.csv')


In [37]:
# 분석할 열들
columns = [
    'w08chronic_a', 'w08chronic_b', 'w08chronic_c',
    'w08chronic_d', 'w08chronic_e', 'w08chronic_f',
    'w08chronic_g', 'w08chronic_h', 'w08chronic_i',
    'w08chronic_k', 'w08chronic_l', 'w08chronic_m'
]

In [38]:
# 2. 데이터 클리닝 - 'w08chronic_m'에서 값이 3인 행 제거
cleaned_data = data[data['w08chronic_m'] != 3]

In [39]:
# 3. 피처 데이터 정의 (모든 chronic 컬럼 제외)
X = cleaned_data.drop(columns=columns)
y_all = cleaned_data[columns]


In [40]:
# 4. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [41]:
# 5. 모델 정의 및 파라미터 그리드
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'MLP': MLPClassifier(max_iter=2000, random_state=42)
}

In [42]:
# RandomizedSearchCV를 위한 파라미터 그리드 설정
param_grids = {
    'Random Forest': {
        'n_estimators': sp_randint(50, 150),
        'max_depth': sp_randint(5, 15),
        'min_samples_split': sp_randint(2, 10),
        'min_samples_leaf': sp_randint(1, 5)
    },
    'Decision Tree': {
        'max_depth': sp_randint(3, 15),
        'min_samples_split': sp_randint(2, 10),
        'min_samples_leaf': sp_randint(1, 5)
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    },
    'MLP': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'activation': ['tanh', 'relu'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001],
        'learning_rate': ['constant', 'adaptive']
    }
}

In [43]:
# 6. 결과 저장을 위한 빈 데이터프레임 생성
result_table = pd.DataFrame(columns=['Target', 'Model', 'Precision', 'Recall', 'F1-Score', 'Accuracy', 'Micro F1', 'Macro F1', 'Weighted F1', 'AUC'])

In [44]:
# 7. 각 타겟 열을 사용한 학습 및 평가
for target_column in columns:
    print(f'\n[{target_column}] 열을 예측:')
    print('==================================')

    y = cleaned_data[target_column].values

    # 데이터 나누기
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # SelectKBest로 피처 선택 (f_classif 사용, 최대 10개의 중요한 피처 선택)
    selector = SelectKBest(f_classif, k=10)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # 타겟 클래스가 두 개 이상이면 AUC 계산을 위해 타겟을 이진화 (One-vs-Rest 방식)
    y_test_bin = label_binarize(y_test, classes=np.unique(y))
    if y_test_bin.shape[1] == 1:  # 클래스가 하나인 경우
        y_test_bin = np.concatenate([1 - y_test_bin, y_test_bin], axis=1)

    # 8. 각 모델에 대해 RandomizedSearchCV를 사용한 학습 및 평가
    for model_name, model in models.items():
        print(f'\n[{model_name}] 모델 평가:')
        print('--------------------------')

        try:
            # RandomizedSearchCV 설정
            random_search = RandomizedSearchCV(model, param_grids[model_name], n_iter=10, scoring='f1_weighted', cv=3, random_state=42, n_jobs=-1)
            
            # 모델 학습
            random_search.fit(X_train_selected, y_train)
            best_model = random_search.best_estimator_

            # 예측
            y_pred = best_model.predict(X_test_selected)
            
            # 확률 예측 값이 있는 경우
            if hasattr(best_model, "predict_proba"):
                y_pred_proba = best_model.predict_proba(X_test_selected)
            else:
                y_pred_proba = np.zeros((len(y_test), len(np.unique(y_test))))  # 확률 값을 지원하지 않는 모델에 대한 처리

            # 정확도 계산
            accuracy = accuracy_score(y_test, y_pred)

            # precision, recall, f1-score 계산 (weighted average)
            precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

            # Micro, Macro, Weighted F1 계산
            micro_f1 = f1_score(y_test, y_pred, average='micro')
            macro_f1 = f1_score(y_test, y_pred, average='macro')
            weighted_f1 = f1_score(y_test, y_pred, average='weighted')

            # AUC 계산 (다중 클래스의 경우 One-vs-Rest 방식 사용)
            try:
                auc = roc_auc_score(y_test_bin, y_pred_proba, multi_class="ovr", average='weighted')
            except ValueError:
                auc = np.nan  # AUC 계산 불가 시 처리

            # 결과 추가
            new_row = pd.DataFrame({
                'Model': [model_name],
                'Target': [target_column],
                'Precision': [precision],
                'Recall': [recall],
                'F1-Score': [f1],
                'Accuracy': [accuracy],
                'Micro F1': [micro_f1],
                'Macro F1': [macro_f1],
                'Weighted F1': [weighted_f1],
                'AUC': [auc]
            })
            result_table = pd.concat([result_table, new_row], ignore_index=True)
        except Exception as e:
            print(f"{model_name} 모델에서 오류 발생: {e}")
            continue



[w08chronic_a] 열을 예측:

[Random Forest] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 86 120 121 122] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw



[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_b] 열을 예측:

[Random Forest] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 86 120 121 122] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw



[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_c] 열을 예측:

[Random Forest] 모델 평가:
--------------------------


...


[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[MLP] 모델 평가:
--------------------------

[w08chronic_m] 열을 예측:

[Random Forest] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 86 120 121 122] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw



[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------


In [45]:
# 9. 결과 테이블 출력
print("\n전체 모델 평가 결과:")
print(result_table)


전체 모델 평가 결과:
          Target          Model  Precision    Recall  F1-Score  Accuracy  \
0   w08chronic_a  Random Forest   0.743987  0.718543  0.713934  0.718543   
1   w08chronic_a  Decision Tree   0.772184  0.730684  0.723281  0.730684   
2   w08chronic_a            SVM   0.773610  0.729581  0.721621  0.729581   
3   w08chronic_a            MLP   0.768633  0.728477  0.721219  0.728477   
4   w08chronic_b  Random Forest   0.680586  0.761589  0.683426  0.761589   
5   w08chronic_b  Decision Tree   0.665517  0.738411  0.685107  0.738411   
6   w08chronic_b            SVM   0.661748  0.757174  0.677564  0.757174   
7   w08chronic_b            MLP   0.689358  0.750552  0.700316  0.750552   
8   w08chronic_c  Random Forest   0.874835  0.923841  0.892459  0.923841   
9   w08chronic_c  Decision Tree   0.879287  0.922737  0.893748  0.922737   
10  w08chronic_c            SVM   0.859611  0.927152  0.892105  0.927152   
11  w08chronic_c            MLP   0.859387  0.923841  0.890449  0.923841  